In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
new_path = '/content/drive/MyDrive/PRIM_HSU'

# Установить новый путь в качестве текущей директории
os.chdir(new_path)

In [2]:
from src.data_processing import HyperspectralImage, PerturbedSVDataPatches, generate_honest_sv_patches
from torch.utils.data import DataLoader
from src.models import Autoencoder
from src.train_utils import train_model
from src.conf import BATCH_SIZE
from src.loss_functions import balance_loss
import matplotlib.pyplot as plt
import torch
from src.metrics import compare_endmembers, compare_abundances
import numpy as np

In [ ]:
samson = HyperspectralImage(image_path="datasets/samson/samson_1.mat",
                           ground_truth_path="datasets/samson/end3.mat",
                           n_bands=156, n_row=95, n_col=95, n_sources=3)

X_1_estim = np.load('results/first_estimation/samson_X_1_estim.npy')
A_1_estim = np.load('results/first_estimation/samson_A_1_estim.npy')
S_1_estim = np.load('results/first_estimation/samson_S_1_estim.npy')

generate_honest_sv_patches(samson, A_1_estim, S_1_estim)

plt.imshow(urban.X[0])
plt.show()

pertubed_sv_image_dataset = PerturbedSVDataPatches()


pertubed_sv_image_dataloader = DataLoader(dataset=pertubed_sv_image_dataset, batch_size=BATCH_SIZE, shuffle=True)
a = next(iter(pertubed_sv_image_dataloader))
print('original_image_dataloader', a["X"].shape)
plt.imshow(a['X'][0][50])
plt.show()

In [ ]:
model = Autoencoder(channel_number=urban.n_bands, source_number=urban.n_sources)
model, loss_train = train_model(model, samson.X,
                                pertubed_sv_image_dataloader,
                                balance_loss, num_epochs=50, n_sources=3, need_plot=True)

In [ ]:
plt.plot(loss_train)

In [ ]:
import torch
torch.save(model.state_dict(), "results/models_weights/9_samson_simple_perturbed_patches.pth")